In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")

In [1]:
import tensorflow as tf

from src.encoders.context_encoder_bert import ContextEncoder

from tensorflow.python import keras
import toml
import json
import pandas as pd

from src.dataset.albert import AlbertDataset

ModuleNotFoundError: No module named 'src'

In [4]:
# Read local `config.toml` file.
config = toml.load('../settings/experiments.toml')
print(config)

{'ALBERT_FINETUNE_SIMPLE': [{'bert_type': 'albert', 'dataset_type': 'clinical', 'finetune_bert': True, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 200}, {'bert_type': 'albert', 'finetune_bert': True, 'dataset_type': 'clinical', 'pct_data': 1, 'augment_pct': 0.5, 'epochs': 200}, {'bert_type': 'albert', 'finetune_bert': True, 'dataset_type': 'clinical', 'pct_data': 1, 'augment_pct': 1, 'epochs': 200}, {'bert_type': 'albert', 'dataset_type': 'fiction', 'finetune_bert': True, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 200}, {'bert_type': 'albert', 'finetune_bert': True, 'dataset_type': 'fiction', 'pct_data': 1, 'augment_pct': 0.5, 'epochs': 200}, {'bert_type': 'albert', 'finetune_bert': True, 'dataset_type': 'fiction', 'pct_data': 1, 'augment_pct': 1, 'epochs': 200}, {'bert_type': 'albert', 'dataset_type': 'wiki', 'finetune_bert': True, 'pct_data': 1, 'augment_pct': 0.1, 'epochs': 200}, {'bert_type': 'albert', 'finetune_bert': True, 'dataset_type': 'wiki', 'pct_data': 1, 'augment_pct': 

In [5]:
# def pool_output(input_tensor):
#     bert_full_output = tf.transpose(input_tensor, [0, 2, 1])
#     bert_pooled_output = tf.reduce_mean(bert_full_output, 2)
#     return bert_pooled_output

In [6]:
# pool_output(tf.constant([[[1,2,3,4], [5,6,7,8]], [[1,2,3,4], [5,6,7,8]]]))

In [7]:
model = ContextEncoder(final_dropout=0.5,
                       dense_neurons=64,
                       bert_trainable=True,
                       bert_type="albert-base-v2")

In [8]:
fake_output = model(tf.constant([[[1,2]],[[3,4]],[[5,6]]])) 

In [9]:
fake_output

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.22578469]], dtype=float32)>

In [10]:
model.summary()

Model: "context_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dense_input_left (Dense)     multiple                  49216     
_________________________________________________________________
dense_input_mid (Dense)      multiple                  49216     
_________________________________________________________________
dense_input_right (Dense)    multiple                  49216     
_________________________________________________________________
dense_output (Dense)         multiple                  193       
_________________________________________________________________
final_dropout (Dropout)      multiple                  0         
Total params: 11,831,425
Trainable params: 11,831,425
Non-trainable params: 0
_______________________________________

In [26]:
dataset = AlbertDataset(dataset_type="clinical",
                       pct_data=0.001,
                       max_segment_length=5,
                       augment_pct=0.001)

In [27]:
sentences, tokenized_sentences, labels = dataset.process()

In [28]:
len(sentences)

9

In [29]:
dataset.format_sentences_tri_input(tokenized_sentences)

(<tf.Tensor: shape=(9, 256), dtype=int32, numpy=
 array([[    2,    53,    16, ...,     0,     0,     0],
        [    2, 18084,    14, ...,     0,     0,     0],
        [    2,   400,    19, ...,     0,     0,     0],
        ...,
        [    2,   235,    31, ...,     0,     0,     0],
        [    2,   273,   468, ...,     0,     0,     0],
        [    2,    21,  1825, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(9, 256), dtype=int32, numpy=
 array([[    2, 18084,    14, ...,     0,     0,     0],
        [    2,   400,    19, ...,     0,     0,     0],
        [    2,   453,   133, ...,     0,     0,     0],
        ...,
        [    2,   273,   468, ...,     0,     0,     0],
        [    2,    21,  1825, ...,     0,     0,     0],
        [    2,    53,    16, ...,     0,     0,     0]])>,
 <tf.Tensor: shape=(9, 256), dtype=int32, numpy=
 array([[    2,   400,    19, ...,     0,     0,     0],
        [    2,   453,   133, ...,     0,     0,     0],
        [    2,   469, 

## Training

In [30]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy')
]

In [31]:
EPOCHS = 1
BATCH_SIZE = 4
dense_output = 128

# balanced = balanced binary crossentropy
checkpoint_filepath = '../models/ALBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                        dataset.dataset_type,                    
                        len(sentences), 
                        dataset.pct_data,
                        dataset.augment_pct)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=False,
    mode="auto",
    save_freq="epoch")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

callbacks = [
#     early_stopping,
    model_checkpoint_callback
]

In [32]:
checkpoint_filepath

'../models/ALBERT/finetune/simple/clinical-9-0.001-pct-0.001-aug/checkpoint'

In [33]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)

In [34]:
try:
    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint available.")

No checkpoint available.


In [35]:
history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                    tf.convert_to_tensor(labels), 
                    epochs=EPOCHS,
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    verbose=1, 
#                     class_weight=class_weight,
                    callbacks=callbacks)

2/2 [==============================] - 42s 16s/step - loss: 0.9206 - accuracy: 0.5000 - val_loss: 0.0343 - val_accuracy: 1.0000


In [38]:
history.history

{'loss': [0.8650528192520142],
 'accuracy': [0.5],
 'val_loss': [0.0343400277197361],
 'val_accuracy': [1.0]}

## Experiment

In [75]:
sys.path.append("../")
from utils.experiments import get_experiments, save_results

In [57]:
# Read local `config.toml` file.
config = get_experiments('ALBERT_FINETUNE_SIMPLE')
config_df = pd.DataFrame.from_dict(config)
config_df

,bert_type,dataset_type,finetune_bert,pct_data,augment_pct,epochs
0,albert,clinical,True,1,0.1,200
1,albert,clinical,True,1,0.5,200
2,albert,clinical,True,1,1.0,200
3,albert,fiction,True,1,0.1,200
4,albert,fiction,True,1,0.5,200
5,albert,fiction,True,1,1.0,200
6,albert,wiki,True,1,0.1,200
7,albert,wiki,True,1,0.5,200
8,albert,wiki,True,1,1.0,200


In [69]:
config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [59]:
for experiment in config:
    bert_type = config['bert_type']
    dataset_type = config['dataset_type']
    finetune_bert = config['finetune_bert']
    pct_data = config['pct_data']
    augment_pct = config['augment_pct']
    epochs = config['epochs']
    print("params:", bert_type, dataset_type, finetune_bert, pct_data, augment_pct, epochs)
    
    # init model
    print("initializing model...")
    model = ContextEncoder(final_dropout=0.5,
                           dense_neurons=64,
                           bert_trainable=finetune_bert,
                           bert_type="albert-base-v2")
    
    # init dataset
    print("initializing dataset...")
    dataset = AlbertDataset(dataset_type=dataset_type,
                           pct_data=pct_data,
                           max_segment_length=5,
                           augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()
    
    # create checkpoint path
    checkpoint_filepath = '../models/ALBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                            dataset_type,                    
                            len(sentences), 
                            pct_data,
                            augment_pct)
    print(checkpoint_filepath)
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=METRICS)
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    # 
    print("starting the training process...")
    history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                        tf.convert_to_tensor(labels), 
                        epochs=EPOCHS,
                        validation_split=0.1,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history
    
    print("saving results...")
    save_results(experiment)

9